In [26]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load local DB helper (avoid installed-package collisions)
import importlib.util, sys
PROJECT_ROOT = os.path.abspath(os.getcwd())
module_path = os.path.join(PROJECT_ROOT, 'db', 'chat_history.py')
spec = importlib.util.spec_from_file_location('local_chat_history', module_path)
local_db = importlib.util.module_from_spec(spec)
spec.loader.exec_module(local_db)
init_db = local_db.init_db
save_message = local_db.save_message
get_history = local_db.get_history
# optional session helpers (may not exist on older DB)
get_session_state = getattr(local_db, 'get_session_state', None)
set_session_state = getattr(local_db, 'set_session_state', None)

load_dotenv()
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
MODEL = "gpt-5-mini"

# Ensure DB exists
init_db()

def detect_incident(user_text):
    prompt = (
        "Classify the type of legal case in ONE WORD only. Possible examples: "
        "civil, criminal, economic, labor, family, inheritance, land, administrative, "
        "serious-criminal, other. Answer strictly one short word."
    )

    messages = [
        {"role": "system", "content": "You are a precise legal-case classifier."},
        {"role": "user", "content": user_text},
        {"role": "system", "content": prompt}
    ]

    resp = client.chat.completions.create(model=MODEL, messages=messages)
    return resp.choices[0].message.content.strip().lower()

def response(user_text, user_id="default_user", history_limit=100, prompt=""):

    save_message(user_id, "user", user_text)

    rows = get_history(user_id, limit=history_limit)

    # 3) Extract only user messages (session memory)
    user_msgs = [r["content"] for r in rows if r["role"] == "user"]
    memory = "\n".join(user_msgs).strip()

    messages = [
        {"role": "system", "content": prompt},
    ]

    if memory:
        messages.append({"role": "system", "content": f"Session memory (user only):\n{memory}"})

    for r in rows[-6:]:   # keep last ~3 turns each side
        messages.append({"role": r["role"], "content": r["content"]})

    resp = client.chat.completions.create(model=MODEL, messages=messages)
    assistant_text = resp.choices[0].message.content.strip()

    save_message(user_id, "assistant", assistant_text)

    return assistant_text

def needs_legal_request(text):
    """Return True when user explicitly asks for legal help/advice (keyword-based)."""
    kws = ["lawyer", "legal", "advice", "律师", "合法的", "建议"]
    t = (text or "").lower()
    return any(k in t for k in kws)


def generate_response(user_text, user_id="default_user", history_limit=100):

    if needs_legal_request(user_text):
        prompt = (
                "User requested legal help. Do NOT give firm legal advice and don't write it. Ask for timeline, sequence, evidence, and names involved. Keep ≤500 chars."
            )
    else:
        prompt = (
                "If they are losing their temper, advise them first. Do NOT give advice and don't write it. Ask ONE concise clarifying question (timeline, whened, evidence, or who) or offer to help find a lawyer if they want. Keep ≤200 chars and end with one open question."
            )

    return response(
        user_text,
        user_id=user_id,
        history_limit=history_limit,
        prompt=prompt
    )


ImportError: attempted relative import with no known parent package

In [25]:
user_text = "I killed someone in self-defense"
incident = detect_incident(user_text)
response1 = generate_response(user_text, user_id="userrrrrrrrrr4")

print(incident)
print(response1)

serious-criminal
I'm sorry you're going through this. Would you like me to help find a lawyer now?


In [20]:
user_text = "i want to know do that in this case"
incident = detect_incident(user_text)
response1 = generate_response(user_text, user_id="userrrrrrrrrr2")

print(incident)
print(response1)

other
I'm sorry you're going through this. I can't give legal advice — would you like help finding a criminal defense lawyer now?


In [21]:
user_text = "I would like to make an appointment with a lawyer"
incident = detect_incident(user_text)
response1 = generate_response(user_text, user_id="userrrrrrrrrr2")

print(incident)
print(response1)

other
I can help arrange an appointment but can’t give legal advice. Please provide: 1) date/time and exact sequence of events; 2) names of the deceased, witnesses, and any officers; 3) what evidence exists (video, photos, medical reports, weapon); 4) whether you were arrested/charged and where; 5) your location and preferred appointment times. Are you safe now? I can help find a criminal defense lawyer.


In [16]:
user_text = "my husband beat me. i want to kill him"
incident = detect_incident(user_text)
response1 = generate_response(user_text, user_id="userrrrrrrrrr1")

print(incident)
print(response1)

serious-criminal
I'm so sorry this happened — that sounds terrifying and overwhelming. I'm here with you and I care about what you're feeling.


In [17]:
user_text = "i want to sue him"
incident = detect_incident(user_text)
response1 = generate_response(user_text, user_id="userrrrrrrrrr1")

print(incident)
print(response1)

civil
I'm so sorry — that sounds terrifying and deeply upsetting. You didn't deserve this. I'm here with you and I care about what you're feeling.


In [22]:
# Print chat history for a user (change `user_id` as needed)
user_id = 'userrrrrrrrrr2'
# Fetch up to 500 messages for this user
history = get_history(user_id, limit=500)

if not history:
    print(f'No history found for user_id: {user_id}')
else:
    for msg in history:
        ts = msg.get('created_at', '')
        role = msg.get('role', '').upper()
        content = msg.get('content', '')
        print(f"[{ts}] {role}: {content}")

[2025-11-14 03:13:35.479] USER: I killed someone in self-defense and need legal advice.
[2025-11-14 03:13:47.154] ASSISTANT: Are you safe now? I can’t give legal advice, but I can help gather facts. Please provide: 1) date/time and exact sequence of events; 2) names of the deceased, witnesses, and any officers; 3) what evidence exists (video, photos, medical reports, weapon); 4) whether you were arrested or charged and where. Need emergency help? I can help find a criminal defense lawyer.
[2025-11-14 03:14:39.157] USER: i want to know do that in this case
[2025-11-14 03:14:52.080] ASSISTANT: I'm sorry you're going through this. I can't give legal advice — would you like help finding a criminal defense lawyer now?
[2025-11-14 03:17:34.974] USER: I would like to make an appointment with a lawyer
[2025-11-14 03:17:44.373] ASSISTANT: I can help arrange an appointment but can’t give legal advice. Please provide: 1) date/time and exact sequence of events; 2) names of the deceased, witnesses,